# NLP - Cleaning the dataset

In [ ]:
# Reading in the data

import pandas as pd
train = pd.read_csv('/Users/helenabelloff/Desktop/NLP/train.csv')
train.head()

In [ ]:
# Now we can remove hyperlinks using some regex code 
# NOTE: we can include some hyperlinks after our initial RF run to see if accuracy improves
# Ex: www.Bloomberg/blah blah, www.bbc/blah because those are probably links to REAL disaster stories

# But let's remove them all for now and see what happens
# ALSO: Let's remove them before tokenization

def hyperlink_remove(text):
    hyperlink_remove = train['text'].str.replace('http\S+|http.\S+', '', case=False)
    return hyperlink_remove

train['hyperlink_remove'] = hyperlink_remove(train['text'])

In [ ]:
# Removing punctuation

import string
string.punctuation

In [ ]:
def remove_punct(text):
    nopunct = "".join([char for char in text if char not in string.punctuation])
    return nopunct

train['text_no_punct'] = train['hyperlink_remove'].apply(lambda x: remove_punct(x))

In [ ]:
# Remove digits

import string

train['text_no_punct'] = train['text_no_punct'].str.replace(r"\d","", regex= True)

In [ ]:
# Text to lower

def text_lower(text):
    text = text.lower()
    return text
train['text_no_punct_lower'] = train['text_no_punct'].apply(lambda x: text_lower(x))

In [ ]:
# Getting rid of accents etc.

import unicodedata
train['text_no_punct_lower'] = train['text_no_punct_lower'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
# Trying to standardize words
from itertools import product
import itertools

train['text_no_punct_lower'] = train['text_no_punct_lower'].apply(lambda x:''.join(''.join(s)[:2] for _, s in itertools.groupby(x)))

In [ ]:
# Tokenizing

import re

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

train['tokenized_text'] = train['text_no_punct_lower'].apply(lambda x: tokenize(x.lower()))

In [ ]:
# Remove stopwords like "the", "for", "are" 
# These don't really do anything for us
# This is a list that I looked up and then I added some stuff after I realized that things like "am" and "pm" 
# wouldn't add anything to the classification analysis

stopword = {'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 
            'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 
            'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 
            'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 
            'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 
            'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 
            'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 
            'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 
            'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 
            'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 
            'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 
            'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 
            'was', 'here', 'than', 'amp', 'wa', 'am', 'pm', 'im', 'leh', 'ind', 'inciweb', 'ina'}

# I've opted to not remove the word "near" because near is a common word found in real disaster tweets 
# (as indicated by my wordclouds)

In [ ]:
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    return text

train['tokenized_nostopwords'] = train['tokenized_text'].apply(lambda x: remove_stopwords(x))

In [ ]:
# Need to remove stems like "ing" and "ly" - destemming

import nltk

port = nltk.PorterStemmer()

def stemming(tokenized_text):
    text = [port.stem(word) for word in tokenized_text]
    return text

train['tokenized_destemmed'] = train['tokenized_nostopwords'].apply(lambda x: stemming(x))

In [ ]:
# Lemmatizing basically does the same thing as stemming, but it takes an informed analysis with 
# the context of the word in mind

# Stemming will just chop off the ending of the word with no context
# Lemmatizing will analyze the root word: Ex: "Entitled", "Entitling" --> "Entitle"

# I will ultimately use the lemmatizer

import nltk

lemmatizer = nltk.WordNetLemmatizer()

def lemmatizing(tokenized_text):
    text = [lemmatizer.lemmatize(word) for word in tokenized_text]
    return text

train['token_lemmatized'] = train['tokenized_nostopwords'].apply(lambda x: lemmatizing(x))

In [ ]:
def join_text(text):
    text = ' '.join(text)
    return text
train['token_lemmatized'] = train['token_lemmatized'].apply(lambda x: join_text(x))